# Installations and Setup

In [ ]:
from google.colab import drive

In [ ]:
# TODO: Mount your Google Drive; this allows the runtime environment to access your drive.
drive.mount('/content/gdrive')

# NOTE: Make sure your path does NOT include a '/' at the end!
base_dir = "/content/gdrive/MyDrive/CS4782_Assignments/a4_release"
sys.path.append(base_dir)
## END TODO

In [ ]:
%pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.assembled import AssembledModel
from models.train_loop import PatchTSTTrainer

# Model Architecture

In [ ]:
patch_tst = AssembledModel()

# Train Model

In [ ]:
trainer = PatchTSTTrainer(patch_tst, "./models/results")
device = "cpu"
dataloader = TSDataLoader("./data/data_files/electricity/electricity.csv", device=device)

# Evaluate Model